In [1]:
import os
os.getcwd()

'f:\\CodingEnvironment\\DPDPTW2F\\data'

In [2]:
cd ..

f:\CodingEnvironment\DPDPTW2F


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
import os
import numpy as np
import csv
from graph.node import Node
from graph.request import Request

def create_requests(file_path, output_path):
    # Read and parse the input file
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    node_list = []
    with open(file_path, 'rt') as f:
        count = 1
        for line in f:
            # Read vehicle parameters from the first line
            if count == 1:
                vehicle_num, vehicle_cap, vehicle_speed = line.split()
                # Adjust vehicle_num as per original logic
                vehicle_num = int(int(vehicle_num) / 2.5)
                vehicle_cap = int(vehicle_cap)
                vehicle_speed = float(vehicle_speed)
            else:
                node_list.append(line.split())
            count += 1

    # Create Node instances from the parsed data
    nodes = [
        Node(
            nid=int(item[0]),
            x=float(item[1]),
            y=float(item[2]),
            demand=float(item[3]),
            ready_time=float(item[4]),
            due_time=float(item[5]),
            service_time=float(item[6]),
            pid=int(item[7]),
            did=int(item[8]),
            time=-1
        ) for item in node_list
    ]

    def create_requests_and_save_to_csv(nodes, filename):
        # Create a lookup dictionary for nodes by their nid for fast access
        node_lookup = {node.nid: node for node in nodes}

        requests = []
        request_id = 1

        # Process nodes to create merged requests
        for node in nodes:
            # Only consider nodes that reference another node via pid or did
            if node.pid != 0 or node.did != 0:
                # Determine pickup node: if pid exists, use the referenced node; otherwise, use current node
                if node.pid != 0 and node.pid in node_lookup:
                    pnode = node_lookup[node.pid]
                else:
                    pnode = node

                # Determine delivery node: if did exists, use the referenced node; otherwise, use current node
                if node.did != 0 and node.did in node_lookup:
                    dnode = node_lookup[node.did]
                else:
                    dnode = node

                # Create a merged request using data from the pickup and delivery nodes
                request = Request(
                    nid=request_id,
                    px=pnode.x,
                    py=pnode.y,
                    dx=dnode.x,
                    dy=dnode.y,
                    pready=pnode.ready_time,
                    pdue=pnode.due_time,
                    dready=dnode.ready_time,
                    ddue=dnode.due_time,
                    demand=node.demand,
                    service_time=node.service_time,
                    time=node.time
                )
                requests.append(request)
                request_id += 1

        # Save the merged requests to a CSV file
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            # Write header
            writer.writerow([
                "nid", "px", "py", "dx", "dy", "pready", "pdue", 
                "dready", "ddue", "demand", "service_time", "time"
            ])
            # Write each request's data
            for req in requests:
                writer.writerow([
                    req.nid, req.px, req.py, req.dx, req.dy, 
                    req.pready, req.pdue, req.dready, req.ddue, 
                    req.demand, req.service_time, req.time
                ])

    # Generate requests and save them to CSV
    create_requests_and_save_to_csv(nodes, output_path)

In [6]:
np.random.seed(0)
for type in [8]:
    for i in range(1, 11):
        for dis in ['LC1', 'LC2', 'LR1', 'LR2', 'LRC1', 'LRC2']:
            file_path = f'data/original/pdp_{type}00/{dis}_{type}_{i}.txt'
            output_path = f'data/merge/{type}00/{dis}_{type}_{i}.csv'
            create_requests(file_path, output_path) 

In [8]:
ls

 Volume in drive F is New Volume
 Volume Serial Number is EC26-6E9F

 Directory of f:\CodingEnvironment\DPDPTW2F

01/02/2025  02:02 AM    <DIR>          .
01/02/2025  02:02 AM    <DIR>          ..
12/30/2024  05:18 PM    <DIR>          algorithm
01/02/2025  01:43 AM    <DIR>          data
01/02/2025  01:12 AM    <DIR>          graph
01/02/2025  02:02 AM                 0 problem.py
               1 File(s)              0 bytes
               5 Dir(s)  62,406,746,112 bytes free
